In [1]:
# enter your params

read_dir = '191641_dna_seq_rep_9025003_zip'
wt_pep = '../../../oligo_design/bglb_model.fa'
#allowed_mutants = 'allowed_mutants.txt' 
sample_list = []

In [2]:
# convert all the reads to FASTQ

from Bio import SeqIO
from glob import glob 

for read in glob( '{}/*ab1'.format( read_dir ) ):
    SeqIO.convert( read, 'abi', read.replace( 'ab1', 'fastq' ), 'fastq' )

ImportError: No module named 'Bio'

In [29]:
# read from file 

my_sample_list_file = 'my_samples.txt'

with open( my_sample_list_file ) as fn:
    sample_list = fn.read().split()

# try get a ist of the samples if we don't already have that 

# if not sample_list:
#     my_glob = '{}/*fastq'
#     print 'No sample list, attempting to find one with the glob "{}"'.format( my_glob ) 
#     my_files = glob( my_glob.format( read_dir ) )
#     #my_samples = map( lambda x: x.split( '/' )[1].split( '_' )[2], my_files ) 
#     sample_list = [ x.split( '/' )[1].split( '_' )[2] for x in my_files ] 
#     print sample_list 

In [30]:
print sample_list

['sample_1-1', 'sample_1-2', 'sample_1-3', 'sample_10-1', 'sample_10-2', 'sample_10-3', 'sample_11-1', 'sample_11-2', 'sample_11-3', 'sample_2-1', 'sample_2-2', 'sample_2-3', 'sample_3-1', 'sample_3-2', 'sample_3-3', 'sample_4-1', 'sample_4-2', 'sample_4-3', 'sample_5-1', 'sample_5-2', 'sample_5-3', 'sample_6-1', 'sample_6-2', 'sample_6-3', 'sample_8-1', 'sample_8-2', 'sample_8-3', 'sample_9-1', 'sample_9-2', 'sample_9-3']


In [35]:
# pair and merge the reads 

from subprocess import call, check_output

expect = 0
for sample in sample_list:
    #genscript
#     fwd = glob( '{}/*_{}_T7.fastq'.format( read_dir, sample ) ) 
#     rev = glob( '{}/*_{}_T7-Ter.fastq'.format( read_dir, sample ) )
#     print len(fwd), len(rev)
    #operon 
    fwd = glob( '{}/{}_T7_*.fastq'.format( read_dir, sample ) ) 
    rev = glob( '{}/{}_T7term_*.fastq'.format( read_dir, sample ) )
    print len( fwd ), len( rev )
    if len( fwd ) == len( rev ) == 1:
        pear_opts = [ '-e', '2' ] 
        cmd = [ 'pear', '-f', fwd[0], '-r', rev[0], '-o', '{}/{}'.format( read_dir, sample ) ] + pear_opts
        call( cmd )
        expect += 1 

1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1


In [36]:
# convert all the assemblies to FASTA

for assembly in glob( '{}/*.assembled.fastq'.format( read_dir ) ):
    SeqIO.convert( assembly, 'fastq', assembly.replace( 'fastq', 'fasta' ), 'fasta' )

In [37]:
# load in allowed mutants for error checking 

with open( allowed_mutants ) as fn:
    allowed_mutants = [ i.strip() for i in fn.readlines() if len(i) > 1 ] 

In [42]:
# diff the assemblies against the wild type protein sequence 

singles = []
for assembly in glob( '{}/*.assembled.fasta'.format( read_dir ) ):
    name = assembly.split( '.' )[0] 
    lines = check_output( [ 'blastx', '-subject', wt_pep, '-query', assembly, '-outfmt', '6 sseq qseq length' ] ).split()
    if lines and lines[0] and lines[1] and lines[2]:
        z = zip( lines[0], lines[1] )
        d = [ '{}{}{}'.format( i, n+1, j ) for n, ( i, j ) in enumerate( z ) if i != j ]
        dstr = '+'.join( d ) 
        if len( d ) == 0: 
            print name, '\twild type'
        elif len( d ) == 1:
            if d[0] in allowed_mutants:
                print name, '\t{}'.format( dstr ) 
                singles.append( ( d[0], name ) )
            else:
                print name, '\tbad alignment'
        elif len( d ) > 1:
            #print name, '\t... ({} mutations)'.format( len( d ) ) 
            print name, '\t{}'.format( dstr ) 

1116776/sample_1-1 	C167F
1116776/sample_1-2 	wild type
1116776/sample_1-3 	wild type
1116776/sample_10-1 	wild type
1116776/sample_10-3 	S16T
1116776/sample_11-2 	H119N+Q134P+R135S+G153C+N157K
1116776/sample_11-3 	H119N
1116776/sample_2-1 	wild type
1116776/sample_3-2 	N293T
1116776/sample_4-2 	L219M
1116776/sample_5-3 	H315D
1116776/sample_6-1 	E177D+E208K
1116776/sample_8-3 	Q19N
1116776/sample_9-1 	E406N


In [39]:
singles

[('C167F', '1116776/sample_1-1'),
 ('S16T', '1116776/sample_10-3'),
 ('H119N', '1116776/sample_11-3'),
 ('N293T', '1116776/sample_3-2'),
 ('L219M', '1116776/sample_4-2'),
 ('H315D', '1116776/sample_5-3'),
 ('Q19N', '1116776/sample_8-3'),
 ('E406N', '1116776/sample_9-1')]

In [40]:
dict( singles ) 

{'C167F': '1116776/sample_1-1',
 'E406N': '1116776/sample_9-1',
 'H119N': '1116776/sample_11-3',
 'H315D': '1116776/sample_5-3',
 'L219M': '1116776/sample_4-2',
 'N293T': '1116776/sample_3-2',
 'Q19N': '1116776/sample_8-3',
 'S16T': '1116776/sample_10-3'}

In [41]:
# results 

picks = dict( singles ) 
l = len( picks ) 
print 'Found {} of {} expected mutants'.format( l, expect ) 

Found 8 of 30 expected mutants
